In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

# Setup logging
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    filename='logs/ingestion_db.log',
    level=logging.DEBUG,
    format="%(asctime)s,%(levelname)s,%(message)s",
    filemode='a'
)

# Create DB engine
engine = create_engine('sqlite:///inventory.db')

def ingest_db_chunked(file_path, table_name, engine, chunksize=10000):
    """
    Ingests large CSVs in chunks into the SQLite database
    """
    try:
        chunk_iter = pd.read_csv(file_path, chunksize=chunksize)
        first_chunk = True

        for i, chunk in enumerate(chunk_iter):
            chunk.to_sql(
                name=table_name,
                con=engine,
                if_exists='replace' if first_chunk else 'append',
                index=False
            )
            first_chunk = False
            logging.info(f'Chunk {i+1} of {table_name} inserted')

    except Exception as e:
        logging.error(f"Error while ingesting {file_path}: {e}")

def load_raw_data():
    """
    Loads all CSVs from the Data folder and ingests them chunk-wise into the DB
    """
    os.makedirs('Data', exist_ok=True)
    start = time.time()
    
    for file in os.listdir('Data'):
        if file.endswith('.csv'):
            file_path = os.path.join('Data', file)
            table_name = os.path.splitext(file)[0]

            logging.info(f'Starting ingestion of {file}')
            ingest_db_chunked(file_path, table_name, engine)
            logging.info(f'Completed ingestion of {file}')

    end = time.time()
    total_time = (end - start) / 60
    logging.info(f'Total ingestion time: {total_time:.2f} minutes')

if __name__ == '__main__':
    load_raw_data()
